# Identifying contamination

It is always a good idea to check that your data is from the species you expect it to be.

## Bactinspector

One useful software application for doing this is bactinspector. Bactinspector.... 

In [ ]:
bactinspector check_species -fq 13681_1#18_1.fastq.gz 

What is the predicted species?

Another useful tool for identifying contamination is [Kraken](https://www.ebi.ac.uk/research/enright/software/kraken).

## Installing kraken
Up until now all the software you required has been available on the virtual machine as it has been installed via conda (a software package manager). Kraken is not available on the virtual machine so for this part of the practical we will attempt to use conda to install kraken:

In [ ]:
conda create -n kraken kraken=x.y

Once installed sucessfully activate the software environment that contains kraken.

In [ ]:
conda activate kraken

The reason we have to activate the environment is that often bioinformatics software has multiple software dependencies so having all software installed centrally in one loaction can cause conflicts. For example if trying to use a software application that relies on python 2 and another software application that relies on python 3 it is impossible to have them exist togther. Therefore we create software environments (or boxes) that conatin only the software and depencies needed and switch between them as needed. Now that we have activated the kraken environment let's use it to look for contamination.

## Setting up a database
To run Kraken you need to either build a database or download an existing one. The standard database is very large (33 GB), but thankfully there are some smaller, pre-built databased available. To download the smallest of them, the 4 GB MiniKraken, run:

In [ ]:
wget https://ccb.jhu.edu/software/kraken/dl/minikraken_20171019_4GB.tgz

This may take some time to run. So skip ahead to the next section on Heterozygous SNPs and come back to this section when the download is cmplete. (If you have trouble installing Kraken or downloading the database then we have pre-generated the kraken report file.) 

Onve the database is downloaded all you need to do is un-tar it:

In [ ]:
tar -zxvf minikraken_20171019_4GB.tgz

This version of the database is constructed from complete bacterial, archaeal, and viral genomes in RefSeq, 

## Running Kraken
To run Kraken, you need to provide the path to the database you just created. By default, the input files are assumed to be in FASTA format, so in this case we also need to tell Kraken that our input files are in FASTQ format, gzipped, and that they are paired end reads:

In [ ]:
kraken --db ./minikraken_20171013_4GB --output kraken_results \
    --fastq-input --gzip-compressed --paired \
    data/13681_1#18_1.fastq.gz data/13681_1#18_2.fastq.gz

The five columns in the file that's generated are:

1. "C"/"U": one letter code indicating that the sequence was either classified or unclassified.
2. The sequence ID, obtained from the FASTA/FASTQ header.
3. The taxonomy ID Kraken used to label the sequence; this is 0 if the sequence is unclassified.
4. The length of the sequence in bp.
5. A space-delimited list indicating the LCA mapping of each k-mer in the sequence.

To get a better overview you can create a kraken report:

In [ ]:
kraken-report --db ./minikraken_20171013_4GB kraken_results > kraken-report

## Looking at the results
Let's have a closer look at the kraken_report for the sample. If for some reason your kraken-run failed there is a prebaked kraken-report at data/kraken-report

In [ ]:
head -n 20 kraken-report

The six columns in this file are:

1. Percentage of reads covered by the clade rooted at this taxon
2. Number of reads covered by the clade rooted at this taxon
3. Number of reads assigned directly to this taxon
4. A rank code, indicating (U)nclassified, (D)omain, (K)ingdom, (P)hylum, (C)lass, (O)rder, (F)amily, (G)enus, or (S)pecies. All other ranks are simply '-'.
5. NCBI taxonomy ID
6. Scientific name

## Exercises
__Q1: What is the most prevalent species in this sample?__

__Q2: Does this match the bactinspector results?__

__Q3: What percentage of reads could not be classified?__  

## Heterozygous SNPs
For bacteria, another thing that you can look at to detect contamination is if there are heterozygous SNPs in your samples. Simply put, if you align your reads to a reference, you would expect any SNPs to be homozygous, i.e. if one read differs from the reference genome, then the rest of the reads that map to that same location will also do so:

__Homozygous SNP__  
Ref:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;CTTGAGACGAAATCACTAAAAAACGTGACGACTTG  
Read1:&nbsp;&nbsp;CTTGAGtCG  
Read2:&nbsp;&nbsp;CTTGAGtCGAAA  
Read3:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;GAGtCGAAATCACTAAAA  
Read4:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;GtCGAAATCA  

But if there is contamination, this may not be the case. In the example below, half of the mapped reads have the T allele and half have the A.

__Heterozygous SNP__  
Ref:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;CTTGAGACGAAATCACTAAAAAACGTGACGACTTG  
Read1:&nbsp;&nbsp;CTTGAGtCG   
Read2:&nbsp;&nbsp;CTTGAGaCGAAA  
Read3:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;GAGaCGAAATCACTAAAA  
Read4:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;GtCGAAATCA  


One way to asses this is to map your reads to a reference genoe and acll variants and use bcftools to count the number of heterzygous SNPs.

## ConFindr

An alternative to counting the number of heterozygous variants is to use a tool called ConFindr. ConFindr is a pipeline that can detect contamination in bacterial NGS data, both between and within species. Instead of looking at SNPs/variants across the whole genome, ConFindr works by looking at conserved core genes - either using rMLST genes (53 genes are known to be single copy and conserved across all bacteria with some known exceptions, which ConFindr handles), or custom sets of genes derived from core-genome schemes. As the genes ConFindr looks at are single copy, any sample that has multiple alleles of one or more gene is likely to be contaminated.

To read more information about ConFindr visit: 

https://olc-bioinformatics.github.io/ConFindr/

Unfortunately we do not have time to run ConFindr here. But some of the automatated high throughput pipelines for mapping and snp calling and genome assembly include QC assment in the preprocessing steps like FastQC, bactinspector and confindr. We will be covering these pipelines later in this course.

In [ ]:
Congratulations! You have reached the end of this tutorial.